# Chaine de traitements

[Version 2025: pour plus de clarté, le TP est divisé en 3]

Une chaine de traitements est composée de différents maillons

<img src="fig/chaine2.png">

> L'impact des pré-traitements sur la performance finale est souvent très important

Nous allons donc étudier les pré-traitement les plus classiques et la manière de les implémenter proprement

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split


# A. Transformation des variables de description

Les données brutes sont souvent incomplètes et bruitées, parfois trop spécifiques etc...
L'exploitation de ces informations requière leur transformation.


## A.1. Enrichissement et transformation des données

### A.1.1. Encodage des variables discrètes

Les variables descriptives sont souvent discrètes... Or les approches de machine learning ne savent pas gérer ces informations là (sauf certains arbres).

1. Charger des données discrètes   
    * prédiction de récidive de cancer
2. Afficher les 3 premières lignes (et bien comprendre qu'on a un problème par rapport aux données manipulées jusqu'ici)
3. Transformer les données

In [ ]:
# récupération de données discrètes
import pandas as pd

filename = "data/breast-cancer.csv"
data = pd.read_csv(filename, header=None).values

Xbrut = data[:,:-1]
Ybrut = data[:,-1:]

print(Xbrut[:3,:], Ybrut[:3])

In [ ]:
# Visualisation (=prise de contact)
# Sur 4 variables (arbitrairement)

plt.figure(figsize=(10,4))
n = 4
for i in range(n):
    ax = plt.subplot(1,n,i+1)
    ax.hist(Xbrut[:,i], bins = len(np.unique(Xbrut[:,i])))


In [ ]:
# transformation des données
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(Xbrut)

print("Catégories: \n",enc.categories_)
X = enc.transform(Xbrut) # transformation des X

enc2 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
Y = enc2.fit_transform(Ybrut)[:,0] # même opération (tout en une ligne) pour les Y

print("Echantillon: \n",X[:3,:], Y[:3])

print("Comparaison des dimensions: \n",Xbrut.shape, X.shape)

In [ ]:
# Analyse et performances

from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# définition du modèle
mod = SVC(kernel="linear")
n_fold = 5
scores = cross_val_score(mod, X, Y, cv=n_fold, scoring='accuracy') # tout est caché dedans :)
print(scores)


In [ ]:
# On sait maintenant qu'il faut se méfier de l'accuracy dans les cas déséquilibré...
# vérification sur la balance des étiquettes 
print(np.histogram(Y,2))

# calcul du score f1
scores = cross_val_score(mod, X, Y, cv=n_fold, scoring='f1') # tout est caché dedans :)
print(scores) # ça va

### A.1.2 Création de nouvelles variables

Sur l'exemples de l'échiquier, si on ajoute un descripteur qui change de valeur sur le modulo 2 de la coordonnées des points... Le problème devient (presque) séparable linéairement.

1. On tente de séparer le problème de base
1. On ajoute les variables et on recommence

In [ ]:
# Génération des données
centers = [[float(i),float(j)] for i in range(5) for j in range(5)]
print(centers)
clusters_std = 0.2
X, y = make_blobs(n_samples=100, centers=centers, cluster_std=clusters_std,  n_features=2,   random_state=0)
y = (y % 2)*2 - 1 # chaque centre = 1 classe => Echiquier binaire

print(X.shape)
plt.figure()
plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
# Construction d'un classifieur linéaire

mod = svm.SVC(kernel="linear")
mod.fit(X,y)

sc = cross_val_score(mod,X,y)
print("Scores de classification (val. croisée x5, mod linéaire): ", sc)

# => Les données ne sont pas séparable linéairemnet !

In [ ]:
# ajout de variables (très utiles)

Xi = np.reshape([(np.floor(i)%2)*2-1 for i in X[:,0]+0.5], (-1,1))
Xj = np.reshape([(np.floor(i)%2)*2-1 for i in X[:,1]+0.5], (-1,1))

Xe = np.concatenate((X,Xi,Xj), axis = 1)

plt.figure()
plt.imshow(Xe[:12,:])
# plt.savefig("fig/checkers_Xe.pdf")

sc = cross_val_score(mod,Xe,y)
print("Scores de classification (val. croisée x5, mod linéaire + données enrichies):", sc)

### A.1.3 Discrétisation de variables continues

Il est parfois difficile de tirer parti des informations continues et plus simple de créer des variables encodant directement l'appartenance à un segment.

e.g. : données étalées entre 1 et 10, avec:
* un peu de points uniformément répartis entre 1 et 5
* beaucoup de points centrés en 7.5 avec une faible dispersion
* beaucoup de points centrés en 9 avec une faible dispersion

A peu près de la forme suivante:
<img src="fig/distrib-pts2.png">

$\Rightarrow$ ce type de distribution destabilise les approches de ML

1. La fonction qui permet de faire ça est: `KBinsDiscretizer`
2. Génerer des données et tester cette fonction
3. [OPT] Réfléchir au pont entre modèle linéaire et arbres

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

# générer des données 1D 
###  TODO  ###

# tester la fonction et afficher la matrice créée

## A.2. Normalisation des données (par colonne)

Soit des données tabulaires classiques:
$$X = \begin{pmatrix}  x_{11}& x_{12} &  \ldots & x_{1d}  \\
x_{21}& x_{22} & \ldots & x_{2d} \\
\vdots& \vdots & \ddots &\vdots \\
x_{n1}& x_{n2} & \ldots & x_{nd}  \\
\end{pmatrix} ,\qquad
Y = \begin{pmatrix}  y_{1} \\
y_{2}\\
\vdots\\
y_{n} \\
\end{pmatrix} ,\qquad y_i\in \mathcal Y
$$

Dans la plupart des cas, le problème vient des $X_j$ :

* qui ne sont pas codés dans les mêmes échelles. e.g. colonne $i$ en $10^{-5}$, colonne $j$ en $10^{-9}$ 
* qui ont simplement un biais trop important, e.g. valeur entre $1050$ et $1070$

Les algorithmes de ML sont souvent destabilités par ces écarts de valeur. Il faut normaliser pour obtenir de bonnes performances

On utilise presque tout le temps une normlisation standard gaussienne consistant à centrer réduire les $X_j \Rightarrow \tilde{X}_j \sim \mathcal N(0,1)$.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler

In [ ]:
import pandas as pd

filename = "data/winequality-red.csv"
data = pd.read_csv(filename)

data.head(10)

In [ ]:
# Comparaison des performances avec et sans normalisation
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

X = data.values[:,:-1]
Y = data.values[:,-1]
# remise en forme des Y entre 0 et nClasses
val = np.unique(Y)
transf = dict(zip(val,np.arange(len(val)))) # mapping [x,y,z] => [0,1,2]
Y = np.vectorize(transf.get)(Y)             # application de la transformation

mod = SVC()
sc = cross_val_score( mod, X, Y, scoring='accuracy')

print("Scores ",sc,sc.mean())

# normalisation des données
scal = StandardScaler()

# 1. Appliquer la transformation (cf doc)

# 2. Calculer la performance

###  TODO  ###

print("Scores sur données normalisées ",sc,sc.mean())


**Question d'ouverture**

L'impact de la normalisation est-il le même sur tous les classifieurs?

Non, évidemment

In [ ]:
import xgboost as xgb # !pip install xgboost # en cas de besoin

bst = xgb.XGBClassifier()
sc = cross_val_score( bst, X, Y, scoring='accuracy')
print("xgboost : ",sc, sc.mean())

sc = cross_val_score( bst, Xn, Y, scoring='accuracy')
print("xgboost (normalisé): ",sc, sc.mean())

## A.3. Normalisation par individu

Sur certaines applications spécifiques comme la classification de signaux ou les données textuelles, on normalise les individus afin de les rendre comparables

* par défaut, un texte de 100 mots n'est pas comparable avec un texte de 1000 mots
* les log à la station chatelet dans une journée de semaine ne sont pas comparables avec un week-end

Les deux normalisations les plus connues (et utilisées) sont:

1. La normalisation probabiliste (les variables de l'individu somme à 1), on fait une hypothèse multinomiale sur les variables
    * pour le texte notamment
1. La normalisation min-max => 0-1
    * souvent pour les signaux
    * Evidemment, en fonction de l'application visée, ça peut être utile ou au contraire néfaste

In [ ]:
import pandas as pd

filename = "data/Beef_TRAIN.tsv"
data = pd.read_csv(filename, header=None, sep='\t')

print(np.unique(data.values[:,0], return_counts=True))

data.head(10)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
X = data.values[:,1:]
Y = data.values[:,0]
# remise en forme des Y entre 0 et nClasses
val = np.unique(Y)
transf = dict(zip(val,np.arange(len(val)))) # mapping [x,y,z] => [0,1,2]
Y = np.vectorize(transf.get)(Y)             # application de la transformation

mod = SVC()
sc = cross_val_score( mod, X, Y, scoring='accuracy')

print("Scores ",sc,sc.mean())

# normalisation des données
scal = MinMaxScaler()
#scal = StandardScaler() # vous pouvez switcher pour voir...
Xn = scal.fit_transform(X)
sc = cross_val_score( mod, Xn, Y, scoring='accuracy')

print("Scores sur données normalisées ",sc,sc.mean())


# Construction du sujet à partir de la correction

In [1]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###